### Import dependencies


In [ ]:


from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PayloadSchemaType, PointStruct, SparseVectorParams, Document, Prefetch, FusionQuery
from qdrant_client import models

import pandas as pd
import openai
import cohere

from dotenv import load_dotenv

import os



### Retrieval

In [ ]:
load_dotenv()
qdrant_client = QdrantClient(url="http://localhost:6333")

In [ ]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=[text],
        model=model,
    )
    return response.data[0].embedding

In [ ]:
def retrieve_data(query, qdrant_client, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-01-hybrid-search",
        prefetch=[
            Prefetch(
                query=query_embedding,
                using="text-embedding-3-small",
                limit=20
            ),
            Prefetch(
                query=Document(
                    text=query,
                    model="qdrant/bm25"
                ),
                using="bm25",
                limit=20
            )
        ],
        query=FusionQuery(fusion="rrf"),
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []
    retrieved_context_ratings = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        retrieved_context_ratings.append(result.payload["average_rating"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "retrieved_context_ratings": retrieved_context_ratings,
        "similarity_scores": similarity_scores,
    }


In [ ]:


query = "Can I get a laptop?"



In [ ]:

results = retrieve_data(query, qdrant_client, k=20)

In [ ]:

results

### Reranking

In [ ]:
## Can work without api_key too.
cohere_client = cohere.ClientV2(api_key=os.environ["CO_API_KEY"])

In [ ]:
to_rerank = results["retrieved_context"]

In [ ]:
results

In [ ]:
response = cohere_client.rerank(
    model="rerank-v4.0-pro",
    query=query,
    documents=to_rerank,
    top_n=20
)


In [ ]:
response

In [ ]:
## Could not figure this out?, Basically we orddered based on relevance score?
## It is creating a reranked_results list with right index ordered by small to large? <Confused>. I think we are using to_rerank to get the actual description.
## and following order of response.results.. Perfect
reranked_results = [to_rerank[result.index] for result in response.results]

In [ ]:


reranked_results



In [ ]:

response = cohere_client.rerank(
    model="rerank-v4.0-fast",
    query=query,
    documents=to_rerank,
    top_n=20
)



In [ ]:


reranked_results = [to_rerank[result.index] for result in response.results]



In [ ]:


reranked_results

